In [1]:


import psycopg2 
import sys,os,shutil
from p3_generatePathPoints import remove_dir,check_dir,generate_osm_routes_main, log_error
from preprocess_csv_data import preprocess_data
import pandas as pd

import time
from datetime import datetime
from annomize import anonymize_column_values

#config.py contains Variables that contains the user credentials to access Twitter API 
from config import connect_str,dbpassword,dbuser,host,database,shp_table_name,column_name,column_name_value



In [2]:
########
TEMP_DIR 		= './output/temp_csv/'
OUTPUT_DIR 		= './output/'
INPUT_DIR 		= './input/'
INPUT_CSV_FILE	= INPUT_DIR +'original_anonymized.csv'
#INPUT_SHP_FILE = ''
INPUT_SHP_FILE = 'gadm36_NPL_shp/gadm36_NPL_2.shp'
PREPROCESSED_CSV_FILE 	= INPUT_DIR+'preprocessed.csv'	
PREPROCESSED_CLIP_FILE  = INPUT_DIR+'preprocessed_clipped.csv'
SAMPLING_PERCENT = 100 # default

In [3]:
#-------- Database Table names
PROBE_TABLE_NAME  	= 'gps_probe'
CLIPPED_PROBE_TABLE = 'gps_probe_clip'
# graphhopper Map-matching
GPX_DIR = 'map-matching-master/matching-web/src/test/resources/target/'
CSV_DIR = 'input/csv/'
RES_CSV_DIR = 'output/res_csv/' # resultant of mapmatching


In [4]:
#--------functions
  
def close_connection(connection):
    
    if connection is not None:
        connection.close()
        print('Connection closed.')


def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:      
        # connect to the PostgreSQL server
        print('\n Database connected...')
        conn = psycopg2.connect(connect_str)
   
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
   
    return conn


In [5]:

def clip_points_within_selected_region(clean_table_name):
	
	drop_table(clean_table_name)# drop table if exists  
	

	print ("\n Clipping Points within selected Region  \n\t Please wait...")
	#sql = " create table "+ clean_table_name +" AS	SELECT o.*	FROM "+PROBE_TABLE_NAME+" o ,"+shp_table_name+" a WHERE ST_DWithin(a.geom::geography, o.geom::geography, 0.01 ) and  a."+column_name+"='"+column_name_value+"';"
	
	sql = "	CREATE TABLE "+ clean_table_name +" AS	SELECT o.*	FROM "+PROBE_TABLE_NAME+" o ,"+shp_table_name+" a WHERE ST_DWithin(a.geom, o.geom, 0.01 ) AND  a."+column_name+"='"+column_name_value+"';"
	
 
	
	print (sql)
	
	conn= connect()
	cur = conn.cursor()
	cur.execute(sql)  
	conn.commit()	 
	cur.close()
	close_connection(conn)
	
	
  
def drop_table(table_name):     
	conn= connect()
	cur = conn.cursor()
	conn.autocommit = True
	drop_sql = "DROP TABLE IF EXISTS "+ table_name
	print (drop_sql)
	cur.execute(drop_sql) 	
	cur.close()
	close_connection(conn)
	
	time.sleep(2) # sleep for 2 seconds
      
      
def create_db_table(table_name):
	#1. remove table if exists, 2. add csv to table, 3.create geom field
	
	
	drop_table(table_name)# drop table if exists
	
	conn= connect()
	cur = conn.cursor()
		
	create_sql = "CREATE TABLE "+ table_name + "	(\
			id integer,\
			ap_id text ,\
			timestamp timestamp without time zone,\
			latitude double precision,\
			longitude double precision,\
			geom geometry(Point,4326)\
		)"
		
	print (create_sql)
	print ("_________________________________________________________")
		
	cur.execute(create_sql) 
	conn.commit()
	
	cur.close()
	close_connection(conn)
	
	
def create_geometry_from_latlon(table_name):
	conn= connect()
	cur = conn.cursor()
	update_sql = "UPDATE "+ table_name +" SET geom = ST_SetSRID(ST_MakePoint(longitude, latitude), 4326);"
	
	print (update_sql)
	print ("_________________________________________________________")
	cur.execute(update_sql)  # executemany
	conn.commit()
	
	cur.close()
	close_connection(conn)



def create_spatial_index(table_name):
	conn= connect()
	cur = conn.cursor() #
	conn.autocommit = True
	sql = "CREATE INDEX ON "+ table_name +" USING GIST(geom);"
	
	print (sql)
	print ("_________________________________________________________")
	cur.execute(sql)  # executemany
	#conn.commit()
	
	cur.close()
	close_connection(conn)



def vaccum_analyze_spatial_index(table_name):
	conn= connect()
	cur = conn.cursor() #
	conn.autocommit = True
	sql = "VACUUM ANALYZE "+ table_name +";"
	
	print (sql)
	print ("_________________________________________________________")
	cur.execute(sql)  # executemany
	#conn.commit()
	
	cur.close()
	close_connection(conn)


def transform_epsg(table_name, col = 'geom', geometry ='Point', epsg=4326):
	conn= connect()
	cur = conn.cursor() #
	conn.autocommit = True
	sql =  "ALTER TABLE "+ table_name +" \
			ALTER COLUMN "+col+" \
			TYPE Geometry("+geometry+", "+str(epsg)+")  USING ST_Transform(geom, "+str(epsg)+");"
	
	print (sql)
	print ("_________________________________________________________")
	cur.execute(sql)  # executemany
	#conn.commit()
	
	cur.close()
	close_connection(conn)



def clip_data_for_selected_region():
	#--- 2.import csv to PostGIS	
	create_db_table(PROBE_TABLE_NAME)# create target table , drop existing table with same name
	csv_2_psql(PREPROCESSED_CSV_FILE, PROBE_TABLE_NAME)
	
	create_geometry_from_latlon(PROBE_TABLE_NAME)
	
	### CREATE INDEX on Geometry column	
	create_spatial_index(PROBE_TABLE_NAME)
	vaccum_analyze_spatial_index(PROBE_TABLE_NAME)
	
	
	## convert to planner projection ( for faster join operation)
	transform_epsg(PROBE_TABLE_NAME, col = 'geom' , geometry= 'Point', epsg=3857)
	
	transform_epsg(shp_table_name, col = 'geom', geometry= 'MultiPolygon', epsg=3857)

	### Create spatial index for shp table and probe table
	start_time =  datetime.now()
	print (start_time)
	
	#--- 3. Clip GPS points within original region	
	clip_points_within_selected_region(CLIPPED_PROBE_TABLE) ### UNCOMMENT
	
	end_time =  datetime.now() 
	time_taken = (end_time - start_time).total_seconds()
	
	log_error('clip_points_within_selected_region() time(sec) '+ str(time_taken), log_file = 'log_DatabaseTime.txt')
	
	
	#--- 4.  save clean_clipped data to csv
	transform_epsg(PROBE_TABLE_NAME, col = 'geom' , geometry= 'Point', epsg=4326)

	psql_2_csv(PREPROCESSED_CLIP_FILE, CLIPPED_PROBE_TABLE)
	
	
def csv_2_psql(csv_file_name, table_name):	
		
	conn = connect()
	cur = conn.cursor()
	conn.autocommit = True
	csv_fr = open(csv_file_name, 'r') 
	cols = csv_fr.readline().strip('\n').split(',')
	cur.copy_from(csv_fr, table_name, sep=',', columns= cols)  
	csv_fr.close()
	cur.close()
	close_connection(conn)
	print("Imported Data from ", csv_file_name, ' to ', table_name)
	print ("_________________________________________________________")
	
	

def psql_2_csv(csv_file_name, table_name):
	
	conn = connect()
	sql_query = "SELECT *  FROM "+ table_name
	df_original = pd.read_sql_query(sql = sql_query, con = conn)
	close_connection(conn)
	df_original.to_csv(csv_file_name,index=False)
	print("Preprocessed and CLipped File saved: ", csv_file_name)
	print ("_________________________________________________________")
			

def preprocess_csv_file(INPUT_CSV_FILE):
	
		
	#---1. preprocess : deduplicate rows, handle ( same ts, diff loc)
	preprocess_data(SAMPLING_PERCENT, INPUT_CSV_FILE,  PREPROCESSED_CSV_FILE)
	
	if shp_table_name != '':
		clip_data_for_selected_region()

	print( " Preprocessing Complete")


def preprocessing_completed(preprocessed_file_name):
	
	if not os.path.isfile(preprocessed_file_name):
		
		print("Please Complete Preprocessing FIRST !")
		
		return False
	
	return True
	
def generate_routes():
		
	#PREPROCESSED_CLIP_FILE
	output_file =  OUTPUT_DIR +"final_csv_4_mobmap_big.csv"
	if shp_table_name == '':
		if preprocessing_completed(PREPROCESSED_CSV_FILE):
			generate_osm_routes_main(PREPROCESSED_CSV_FILE,output_file)		
	else:
		if preprocessing_completed(PREPROCESSED_CLIP_FILE):
			generate_osm_routes_main(PREPROCESSED_CLIP_FILE,output_file)
		
	print(" Route Generation Complete. Check " + OUTPUT_DIR )


      
def select_csv_file(input_file):
	
	#input_file = filedialog.askopenfilename()
	filename, file_extension = os.path.splitext(input_file)
	command =''
	if file_extension == '.csv':
		#shutil.copy(input_file,INPUT_CSV_FILE)
		anonymize_column_values( 'ap_id', input_file, INPUT_CSV_FILE)
		#print(" CSV file location saved "  )
	
	else:
		
		print(" Please select proper csv file " )
	
	print ('File Obtained: ',input_file)

################

def select_shp_file(INPUT_SHP_FILE):

	if INPUT_SHP_FILE != '':
		drop_table(shp_table_name)# drop table if exists  
		create_table_command = 'shp2pgsql -I -s 4326 '+INPUT_SHP_FILE+'  '+ shp_table_name +' | PGPASSWORD='+dbpassword+' psql -d '+database+' -h '+host+' -U '+dbuser+' '
		print (create_table_command)
		os.system(create_table_command)
	
		print(" Shapefile imported to PostGIS " ) 
	
	

        
def select_shp_file_BAK(INPUT_SHP_FILE):
	
	#INPUT_SHP_FILE = filedialog.askopenfilename()
	

	filename, file_extension = os.path.splitext(INPUT_SHP_FILE)
	if file_extension == '.shp':
		drop_table(shp_table_name)# drop table if exists  
		create_table_command = 'shp2pgsql -I -s 4326 '+INPUT_SHP_FILE+'  '+ shp_table_name +' | PGPASSWORD='+dbpassword+' psql -d '+database+' -h '+host+' -U '+dbuser+' '
		print (create_table_command)
		os.system(create_table_command)
	
		print(" Shapefile imported to PostGIS " ) 
	
	else:
		print(" Please select proper Shapefile " )
   

In [6]:
	    		
#################################################    

# remove old files
remove_dir(OUTPUT_DIR)
remove_dir(INPUT_DIR)
remove_dir(GPX_DIR)
remove_dir(CSV_DIR)
remove_dir(RES_CSV_DIR)


# create necessary directories
check_dir(INPUT_DIR)
check_dir(OUTPUT_DIR)
check_dir(TEMP_DIR)
check_dir(GPX_DIR)
check_dir(CSV_DIR)
check_dir(RES_CSV_DIR)

### RUN PROGRAM

In [7]:

select_shp_file(INPUT_SHP_FILE)



 Database connected...
DROP TABLE IF EXISTS gadm36_NPL_2
Connection closed.
shp2pgsql -I -s 4326 gadm36_NPL_shp/gadm36_NPL_2.shp  gadm36_NPL_2 | PGPASSWORD=postgres123 psql -d mobility -h localhost -U postgres 
 Shapefile imported to PostGIS 


In [8]:
# read input shapefile (OPTIONAL)
input_csv_file = 'gps_probe_Nepal.csv'
select_csv_file(input_csv_file)

File Obtained:  gps_probe_Nepal.csv


In [9]:
# preprocess_csv_file

preprocess_csv_file(INPUT_CSV_FILE)

3  csv prepared:  input/csv/

completed:  java -jar matching-web/target/graphhopper-map-matching-web-1.0-SNAPSHOT.jar import map-data/nepal-latest.osm.pbf
Current Working Directory  /home/bidur/GPSProbeGeneration/map-matching-master

completed:  java -jar matching-web/target/graphhopper-map-matching-web-1.0-SNAPSHOT.jar match matching-web/src/test/resources/target/*.gpx
convert_resgpx2csv output/res_csv/
map-matching-master/matching-web/src/test/resources/target/*.res.gpx
8409248386118 ,completed:  output/res_csv/8409248386118_res.csv
5609555514177 ,completed:  output/res_csv/5609555514177_res.csv
9236094972024 ,completed:  output/res_csv/9236094972024_res.csv
./input/preprocessed.csv

 Database connected...
DROP TABLE IF EXISTS gps_probe
Connection closed.

 Database connected...
CREATE TABLE gps_probe	(			id integer,			ap_id text ,			timestamp timestamp without time zone,			latitude double precision,			longitude double precision,			geom geometry(Point,4326)		)
_______________________

In [10]:
generate_routes()

Input for ROuting: ./input/preprocessed_clipped.csv
----------------------5609555514177----------------------------
0 Processing ...... 5609555514177,364-365 
points count: 3
1 Processing ...... 5609555514177,365-366 
points count: 3
2 Processing ...... 5609555514177,366-367 
points count: 2
3 Processing ...... 5609555514177,367-368 
points count: 3
4 Processing ...... 5609555514177,368-369 
points count: 3
5 Processing ...... 5609555514177,369-370 
points count: 3
6 Processing ...... 5609555514177,370-371 
points count: 5
7 Processing ...... 5609555514177,371-374 
	(STAYPOINT: same location -> different timestamp.) 5609555514177,371-374   Skipping...
8 Processing ...... 5609555514177,374-375 
points count: 4
9 Processing ...... 5609555514177,375-376 
points count: 2
10 Processing ...... 5609555514177,376-377 
points count: 4
11 Processing ...... 5609555514177,377-378 
points count: 4
12 Processing ...... 5609555514177,378-379 
points count: 2
13 Processing ...... 5609555514177,379-380

points count: 2
133 Processing ...... 5609555514177,515-516 
points count: 4
134 Processing ...... 5609555514177,516-517 
points count: 2
135 Processing ...... 5609555514177,517-520 
points count: 3
136 Processing ...... 5609555514177,520-521 
points count: 2
137 Processing ...... 5609555514177,521-522 
points count: 4
138 Processing ...... 5609555514177,522-523 
points count: 3
139 Processing ...... 5609555514177,523-524 
points count: 2
140 Processing ...... 5609555514177,524-525 
points count: 2
141 Processing ...... 5609555514177,525-526 
points count: 3
142 Processing ...... 5609555514177,526-527 
points count: 4
143 Processing ...... 5609555514177,527-528 
points count: 3
144 Processing ...... 5609555514177,528-529 
points count: 3
145 Processing ...... 5609555514177,529-530 
points count: 5
146 Processing ...... 5609555514177,530-531 
points count: 2
147 Processing ...... 5609555514177,531-532 
points count: 4
148 Processing ...... 5609555514177,532-533 
points count: 2
149 Proc